<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yfi
import pandas as pd
import asyncio

In [2]:
def proccess_data(df):
    # Ensure the input is a DataFrame
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    proccessed_data = pd.DataFrame()

    # Calculate the range
    proccessed_data['range'] = df['High'] - df['Low']

    stdev = proccessed_data['range'].std()

    # Calculate the percentage of the range that the closing price represents
    proccessed_data['close_percentage'] = (df['Close'] - df['Low']) / proccessed_data['range']

    proccessed_data['range'] = proccessed_data['range'] / stdev

    # Calculate the difference between the closing and opening prices
    proccessed_data['price_difference'] = (df['Close'] - df['Open']) / stdev

    return proccessed_data

In [3]:
async def fetch_ticker_data(ticker: str, start_date: str, end_date: str, interval: str):
    try:
        data = yfi.download(ticker, start=start_date, end=end_date, interval=interval)
        proccessed_data = proccess_data(data)
        return proccessed_data

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

async def fetch_all_tickers(ticker_list: list,start_date: str,end_date: str, interval: str, max_concurrency:int=50):
    tasks = [fetch_ticker_data(ticker,start_date=start_date,end_date=end_date,interval=interval) for ticker in ticker_list]
    return await asyncio.gather(*tasks[:max_concurrency])

In [4]:
start_date = '2023-01-01'
end_date = '2023-10-22'
interval='60m'

# List of forex tickers    # List of forex tickers
forex_tickers = ["EURUSD=X", "USDEUR=X","GBPUSD=X", 'USDGBP=x', "USDJPY=X","JPYUSD=X", "AUDUSD=X" , "USDAUD=X"]  # Example tickers

# Fetch data concurrently
results = await fetch_all_tickers(forex_tickers,start_date=start_date,end_date=end_date,interval=interval)

combined_df = pd.concat(results, axis=0, ignore_index=True)
combined_df.dropna(inplace=True)
target = combined_df['price_difference'][1:]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
combined_df = combined_df.iloc[:-1]
print(combined_df)
print(target)

          range  close_percentage  price_difference
0      1.364065          0.818044          0.868053
1      1.240482          0.499688         -0.372299
2      0.867279          1.000000          0.000000
3      0.868053          1.000000          0.372170
4      1.364065          0.181672         -0.743954
...         ...               ...               ...
39909  1.264080          1.000000          0.280154
39910  1.077936          0.326980          0.028981
39911  0.994019          0.528793         -0.468389
39912  0.863242          0.385251         -0.212170
39913  0.694545          0.673344          0.429098

[39675 rows x 3 columns]
1       -0.372299
2        0.000000
3        0.372170
4       -0.743954
5       -0.494980
           ...   
39910    0.028981
39911   -0.468389
39912   -0.212170
39913    0.429098
39914   -0.164084
Name: price_difference, Length: 39675, dtype: float64


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GaussianNoise
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from sklearn.model_selection import train_test_split

model = Sequential([
    GaussianNoise(0.05),
    Dense(64, activation='gelu'),
    Dense(32, activation='gelu'),
    Dense(12, activation='gelu'),
    Dense(1)
])


huber_loss = Huber(delta=1.3)
model.compile(optimizer=Adam(), loss=huber_loss)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(combined_df, target, test_size=0.2, random_state=42)

In [8]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
794/794 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.4495 - val_loss: 0.4395
Epoch 2/100
794/794 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.4436 - val_loss: 0.4395
Epoch 3/100
794/794 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.4408 - val_loss: 0.4391
Epoch 4/100
794/794 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.4483 - val_loss: 0.4393
Epoch 5/100
794/794 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.4410 - val_loss: 0.4389
Epoch 6/100
794/794 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.4441 - val_loss: 0.4392
Epoch 7/100
794/794 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4423 - val_loss: 0.4400
Epoch 8/100
794/794 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.4374 - val_loss: 0.4395
Epoch 9/100
794/794 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.4430 - val_loss: 0.4391
Epoch 10/100
794/794 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.4346 - val_loss: 0.4390
Epoch 11/100
794/794 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.4409 - val_loss: 0.4390
Epoch 12/100
794/794 ━━━━━━━━━━━━━━━━━━━━

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-1a4757c3abc7>", line 1, in <cell line: 1>
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py", line 317, in fit
    callbacks.on_train_batch_begin(step)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/callback_list.py", line 98, in on_train_batch_begin
    def on_train_batch_begin(self, batch, logs=None):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/cor

TypeError: object of type 'NoneType' has no len()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()